# Import augmented dataset

In [ ]:
import os
import pandas as pd
import pprint

path = os.getcwd()
csv_input_path = os.path.dirname(path) + "/Doc_Panthera_Augmented/augmented_dataset_final_outputs.csv"
csv_input_path

# Read the CSV file into a DataFrame
data_df = pd.read_csv(csv_input_path, encoding='utf-8')

# Display the first few rows of the DataFrame to check the contents
display(data_df)

# Clean data and Recursive chunking

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(data_df, page_content_column="Text")
docs_data = loader.load()
docs_data[0]

In [ ]:
import importlib
import Data_preprocessing
importlib.reload(Data_preprocessing)

# Initialize the Preprocessing object
preprocessing = Data_preprocessing.Preprocessing()

# Iterate through each document in docs_data and clean the text
for doc in docs_data:
    cleaned_content = preprocessing.clean_text_template(doc.page_content)
    doc.page_content = cleaned_content

pprint.pprint(docs_data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split
# Possible improvements - future hypertuning of chunk_size and chunk_overlap to improve results and try different slitters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=500)

In [ ]:
docs_processed = []
chunk_number = 1  # Initialize chunk number

for doc in docs_data:
    # Split the document into chunks
    split_docs = text_splitter.split_documents([doc])
    
    # Add chunk number as metadata to each split document
    for split_doc in split_docs:
        split_doc.metadata['chunk_number'] = chunk_number
        docs_processed.append(split_doc)
        chunk_number += 1  # Increment chunk number for the next chunk

# Print the first 6 processed documents and their count
pprint.pprint(docs_processed[0:6])
print(len(docs_processed))


# Set up the generation of synthetic data

In [ ]:
from huggingface_hub import InferenceClient
import json

token_pro = os.getenv('HUGGINGFACE_TOKEN')
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
    token=token_pro
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1028},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]

In [ ]:
QA_generation_prompt = """
Il tuo compito è scrivere una domanda e una risposta in Italiano dato il contesto testuale della documentazione del software aziendale. 
La tua domanda deve essere rispondibile con un'informazione specifica dal contesto. 
Se nel contesto ci sono errori grammaticali o morfologici, correggili nell'output fornito.
La tua domanda deve essere formulata nello stesso stile delle domande che gli utenti potrebbero porre ad un helpdesk che si occupa di assistenza clienti
per un software aziendale. 

Nella generazione della domanda non devi fare riferimento direttamente alle descrizioni delle immagini 
e non devi utilizzare informazioni relative a schermate, finestre e testate di tabelle presenti nelle descrizioni delle immagini. 
La tua domanda NON deve menzionare frasi come "secondo il passaggio", "nel contesto", "nella schermata" o "perché l'immagine mostra". 
Non fare domande che chiedano quanti elementi sono visualizzati o che riguardano dati e valori presi da descrizioni che possono derivare da immagini. 

Evita di fare riferimento alle descrizioni delle immagini o di usare dettagli visivi nella generazione della domanda, ma utilizza queste informazioni se necessario solo per la risposta,
considerando che la risposta deve guidare l'utente nella risoluzione delle problematiche e dei quesiti posti nella domanda.
Puoi formulare domande come: "Ho un errore", "Come posso risolvere il problema", o "Come posso sistemare questa situazione?".
La risposta dovrà fare riferimento alle informazioni fornite nel contesto, citando correttamente flag, nomi o altre informazioni pertinenti,
senza mai rivelare esempi o dati sensibili.

Domanda e risposta devono essere generate in Italiano.

Fornisci la tua risposta come segue:

Output:::
Domanda: (la tua domanda)
Risposta: (la tua risposta alla domanda)

Ora ecco il contesto:

Contesto: {context}\n
Output:::"""

In [ ]:
import random
document = random.sample(docs_processed, 1)
pprint.pprint(document)
for sampled_document in document:
    pprint.pprint(call_llm(llm_client, QA_generation_prompt.format(context=sampled_document.page_content)))

# Choose Generation method

## Generate QA using only a sampled chunk each time

In [ ]:
# GENERATE DATASET USING ONLY SAMPLED CHUNK
import random
from tqdm.auto import tqdm

N_GENERATIONS = 100

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Domanda: ")[-1].split("Risposta: ")[0]
        answer = output_QA_couple.split("Risposta: ")[-1]

        outputs.append(
            {
                "question": question,
                "answer": answer,
                "context": sampled_context.page_content,
                "source_doc": sampled_context.metadata["FileName"],
                "chunk_num": sampled_context.metadata["chunk_number"]
            }
        )
    except:
        continue

## Generate QA testset using not only a sampled chunk, but also the adjacent chunks

In [ ]:
# GENERATE DATASET USING NOT ONLY THE SAMPLED CHUNK, BUT ALSO THE ADJACENT ONES
import random
from tqdm.auto import tqdm

N_GENERATIONS = 200

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []

def get_adjacent_chunks(sampled_chunk, docs, radius=1):
    """
    Get adjacent chunks within a given radius for the same document.
    """
    source_doc = sampled_chunk.metadata["FileName"]
    chunk_num = sampled_chunk.metadata["chunk_number"]
    
    # Filter all chunks belonging to the same source_doc
    doc_chunks = [doc for doc in docs if doc.metadata["FileName"] == source_doc]
    
    # Sort the chunks by their chunk_number
    doc_chunks.sort(key=lambda x: x.metadata["chunk_number"])
    
    # Identify adjacent chunks
    start = max(chunk_num - radius, 0)
    end = chunk_num + radius + 1  # Include current and adjacent chunks
    return [chunk for chunk in doc_chunks if start <= chunk.metadata["chunk_number"] < end]

for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Get the sampled chunk along with its adjacent chunks
    adjacent_chunks = get_adjacent_chunks(sampled_context, docs_processed, radius=1)
    
    # Combine contents of the adjacent chunks
    combined_context = " ".join(chunk.page_content for chunk in adjacent_chunks)

    # Record all chunk numbers used
    chunk_numbers = [chunk.metadata["chunk_number"] for chunk in adjacent_chunks]
    
    # Generate QA couple using the combined context
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=combined_context))
    
    try:
        question = output_QA_couple.split("Domanda: ")[-1].split("Risposta: ")[0]
        answer = output_QA_couple.split("Risposta: ")[-1]

        outputs.append(
            {
                "question": question,
                "answer": answer,
                "context": combined_context,
                "source_doc": sampled_context.metadata["FileName"],
                "chunk_num": chunk_numbers
            }
        )
    except Exception as e:
        print(f"Error processing chunk {sampled_context.metadata}: {e}")
        continue

## Generate QA testset, using more than a chunk and adjacent ones sampled from the same document

In [ ]:
# GENERATE DATASET USING NOT ONLY THE SAMPLED CHUNK, BUT ALSO OTHER CHUNKS SAMPLED RANDOMLY FROM THE SAME DOCUMENT
def get_random_chunks(sampled_chunk, docs, n_random=2, radius=1):
    """
    Get n_random chunks from the same document (excluding the sampled chunk and its adjacent ones),
    along with their adjacent chunks, ensuring no duplicate chunk numbers.
    """
    source_doc = sampled_chunk.metadata["FileName"]
    chunk_num = sampled_chunk.metadata["chunk_number"]
    
    # Filter all chunks belonging to the same source_doc
    doc_chunks = [doc for doc in docs if doc.metadata["FileName"] == source_doc]
    
    # Exclude the sampled chunk and its adjacent chunks
    start = max(chunk_num - radius, 0)
    end = chunk_num + radius + 1
    excluded_chunks = {chunk.metadata["chunk_number"] for chunk in doc_chunks if start <= chunk.metadata["chunk_number"] < end}
    available_chunks = [chunk for chunk in doc_chunks if chunk.metadata["chunk_number"] not in excluded_chunks]
    
    # Randomly sample chunks
    sampled_random_chunks = random.sample(available_chunks, min(n_random, len(available_chunks)))
    
    # Include adjacent chunks for each randomly sampled chunk and remove duplicates
    all_random_chunks = []
    unique_chunk_nums = set()  # Track unique chunk numbers

    for chunk in sampled_random_chunks:
        # Get adjacent chunks for the randomly sampled chunk
        adjacent_chunks = get_adjacent_chunks(chunk, docs, radius)
        
        # Add chunks to the result only if their chunk_num is unique
        for adj_chunk in adjacent_chunks:
            if adj_chunk.metadata["chunk_number"] not in unique_chunk_nums:
                all_random_chunks.append(adj_chunk)
                unique_chunk_nums.add(adj_chunk.metadata["chunk_number"])
    
    return all_random_chunks


In [ ]:
# GENERATE DATASET USING NOT ONLY THE SAMPLED CHUNK, BUT ALSO THE ADJACENT ONES
import random
from tqdm.auto import tqdm

N_GENERATIONS = 200

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []

for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    try:
        # Get the sampled chunk along with its adjacent chunks
        adjacent_chunks = get_adjacent_chunks(sampled_context, docs_processed, radius=1)
        
        # Get additional random chunks (and their adjacent chunks) from the same document
        random_chunks = get_random_chunks(sampled_context, docs_processed, n_random=2, radius=1)
        
        # Combine contents of all chunks
        combined_chunks = adjacent_chunks + random_chunks
        combined_context = " ".join(chunk.page_content for chunk in combined_chunks)

        # Record all chunk numbers used
        chunk_numbers = [chunk.metadata["chunk_number"] for chunk in combined_chunks]
        
        # Generate QA couple using the combined context
        output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=combined_context))
        
        question = output_QA_couple.split("Domanda: ")[-1].split("Risposta: ")[0]
        answer = output_QA_couple.split("Risposta: ")[-1]

        outputs.append(
            {
                "question": question,
                "answer": answer,
                "context": combined_context,
                "source_doc": sampled_context.metadata["FileName"],
                "chunk_num": chunk_numbers
            }
        )
    except Exception as e:
        print(f"Error processing chunk {sampled_context.metadata}: {e}")
        continue

# Critique agents

In [ ]:
question_groundedness_critique_prompt = """
Sarà fornito un contesto e una domanda.
Il tuo compito è di fornire una valutazione per indicare quanto bene si possa rispondere in modo univoco alla domanda data con il contesto fornito.
Dai la tua risposta su una scala da 1 a 5, dove 1 significa che la domanda non è affatto rispondibile con il contesto, 
e 5 significa che la domanda è chiaramente e univocamente rispondibile con il contesto.

Fornisci la tua risposta esattamente nel seguente formato:

Output:::
Valutazione totale: (il tuo punteggio, come numero tra 1 e 5)
Output:::

Ora ecco la domanda e il contesto.

Domanda: {question}
Contesto: {context}

Output:::
"""

question_relevance_critique_prompt = """
Ti sarà fornita una domanda.
Il tuo compito è di fornire una "valutazione totale" che rappresenti quanto utile possa essere questa domanda per gli utenti che chiedono assistenza all'help desk riguardo a specifiche funzionalità
del software gestionale e la relativa documentazione.
Dai la tua risposta su una scala da 1 a 5, dove 1 significa che la domanda non è per nulla utile, e 5 significa che la domanda è estremamente utile.

Fornisci la tua risposta esattamente nel seguente formato:

Output:::
Valutazione totale: (il tuo punteggio, come numero tra 1 e 5)
Output:::

Ora ecco la domanda.

Domanda: {question}

Output:::
"""

question_standalone_critique_prompt = """
Ti sarà fornita una domanda.
Il tuo compito è di fornire una "valutazione totale" che rappresenti quanto questa domanda sia indipendente dal contesto.
Dai la tua risposta su una scala da 1 a 5, dove 1 significa che la domanda dipende da informazioni aggiuntive per essere compresa, e 5 significa che la domanda ha senso da sola.
Ad esempio, se la domanda si riferisce a un contesto particolare, come "nel contesto" o "nel documento", la valutazione deve essere 1.
Le domande possono contenere termini tecnici o acronimi e ricevere comunque una valutazione di 5: deve semplicemente essere chiaro per un operatore con accesso alla documentazione di cosa tratta la domanda.

Fornisci la tua risposta esattamente nel seguente formato:

Output:::
Valutazione totale: (il tuo punteggio, come numero tra 1 e 5)
Output:::

Ora ecco la domanda.

Domanda: {question}

Output:::
"""

image_relevance_critique_prompt = """
Ti sarà fornita una domanda.
Il tuo compito è di valutare se la domanda riguarda entità come immagini, schermate, finestre o testate di tabelle che appaiono nelle descrizioni delle immagini.
Inoltre, valuta se la domanda potrebbe contenere dati sensibili derivanti dalle descrizioni delle immagini, come informazioni su schermate, immagini o tabelle.
Se la domanda è probabilmente derivata da un testo che descrive il contenuto di un'immagine, restituisci un punteggio di 0.
Se la domanda non riguarda le immagini o non contiene dati sensibili, restituisci un punteggio di 1.

Fornisci la tua risposta esattamente nel seguente formato:

Output:::
Valutazione totale: (il tuo punteggio, che può essere 0 o 1)
Output:::

Ora ecco la domanda.

Domanda: {question}

Output:::
"""


In [ ]:
import time
import re
print("Generating critique for each QA couple...")

for output in tqdm(outputs):
    time.sleep(1)
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        )
    }
    
    # Initialize scores with None as default values
    output.update({
        "groundedness_score": None,
    })

    # Example code with regex substitution
    for criterion, evaluation in evaluations.items():
    
        # Use regex to find the score following "Valutazione totale:"
        match = re.search(r"Valutazione totale:\s*(\d+)", evaluation)
    
        # Extract the score if the match is found, else set it to a default value (e.g., 0 or None)
        score = int(match.group(1)) if match else 0
    
        output.update(
            {
                f"{criterion}_score": score
            }
        )

In [ ]:
print("Generating critique for each QA couple...")

for output in tqdm(outputs):
    time.sleep(1)
    evaluations = {
        "standalone": call_llm(
            llm_client,  
            question_relevance_critique_prompt.format(question=output["question"]),
        )
    }
    
    # Initialize scores with None as default values
    output.update({
        "standalone_score": None,
    })

    # Example code with regex substitution
    for criterion, evaluation in evaluations.items():
    
        # Use regex to find the score following "Valutazione totale:"
        match = re.search(r"Valutazione totale:\s*(\d+)", evaluation)
    
        # Extract the score if the match is found, else set it to a default value (e.g., 0 or None)
        score = int(match.group(1)) if match else 0
    
        output.update(
            {
                f"{criterion}_score": score
            }
        )

In [ ]:
print("Generating critique for each QA couple...")

for output in tqdm(outputs):
    time.sleep(1)
    evaluations = {
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        )
    }
    
    # Initialize scores with None as default values
    output.update({
        "relevance_score": None,
    })

    # Example code with regex substitution
    for criterion, evaluation in evaluations.items():
    
        # Use regex to find the score following "Valutazione totale:"
        match = re.search(r"Valutazione totale:\s*(\d+)", evaluation)
    
        # Extract the score if the match is found, else set it to a default value (e.g., 0 or None)
        score = int(match.group(1)) if match else 0
    
        output.update(
            {
                f"{criterion}_score": score
            }
        )

In [ ]:
import time
print("Generating critique for each QA couple...")

for output in tqdm(outputs):
    time.sleep(1)
    evaluations = {
        "image_relevance": call_llm(
            llm_client,
            image_relevance_critique_prompt.format(question=output["question"]),
        )
    }
    
    # Initialize scores with None as default values
    output.update({
        "image_relevance_score": None,
    })

    # Example code with regex substitution
    for criterion, evaluation in evaluations.items():
    
        # Use regex to find the score following "Valutazione totale:"
        match = re.search(r"Valutazione totale:\s*(\d+)", evaluation)
    
        # Extract the score if the match is found, else set it to a default value (e.g., 0 or None)
        score = int(match.group(1)) if match else 0
    
        output.update(
            {
                f"{criterion}_score": score
            }
        )

In [ ]:
import pandas as pd
import datasets

pd.set_option("display.max_colwidth", None)

# Create DataFrame after ensuring all columns are initialized
generated_questions = pd.DataFrame.from_dict(outputs)

# Calculate the average score across the three columns
generated_questions["average_score"] = (
    generated_questions["groundedness_score"] 
    + generated_questions["relevance_score"]
    + generated_questions[ "standalone_score"]
)/3

# Filter to keep rows where the average score is greater than 4 and image_relevance_score equals 1
generated_questions = generated_questions.loc[
    (generated_questions["average_score"] > 4) & 
    (generated_questions["image_relevance_score"] == 1) &
    (generated_questions["groundedness_score"] > 4)
]

print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
            "average_score"
        ]
    ]
)

# Create the dataset from filtered DataFrame
new_eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

In [ ]:
display(new_eval_dataset)

In [ ]:
from datasets import Dataset
import pandas as pd

# Load existing dataset
existing_dataset = Dataset.load_from_disk("eval_dataset_random_chunks")
old_elem = pd.DataFrame(existing_dataset)
new_elem = pd.DataFrame(new_eval_dataset)

# Concatenate the old and new DataFrames
combined_df = pd.concat([old_elem, new_elem], ignore_index=True)

# Identify unhashable columns (columns with lists, dicts, etc.)
unhashable_cols = [
    col for col in combined_df.columns 
    if combined_df[col].apply(lambda x: isinstance(x, (list, dict))).any()
]

# Separate the unhashable columns
hashable_df = combined_df.drop(columns=unhashable_cols)
unhashable_df = combined_df[unhashable_cols]

# Drop duplicates on the hashable part and keep the corresponding indices
deduplicated_hashable_df = hashable_df.drop_duplicates()
deduplicated_indices = deduplicated_hashable_df.index

# Use the indices to filter the unhashable part
deduplicated_unhashable_df = unhashable_df.loc[deduplicated_indices]

# Combine the deduplicated hashable and unhashable parts
combined_df = pd.concat(
    [deduplicated_hashable_df.reset_index(drop=True), deduplicated_unhashable_df.reset_index(drop=True)],
    axis=1
)

# Convert the combined DataFrame back to a Dataset
combined_dataset = Dataset.from_pandas(combined_df)
combined_dataset

In [ ]:
combined_dataset.save_to_disk("eval_dataset_random_chunks")